# Sleepwalk on Colab test drive

- Reconstrue
- Link back to Readme.md with `Open in Colab`s

This notebook simply tests if Sleepwalk can be deployed on Colab, hopefully even viewing the dynamic output in Colab or as a fallback simply generate the web view statics pages and copy them to some storage.

## Configure platform

This notebook started from [a demo R-on-Colab notebook](https://github.com/IRkernel/IRkernel/blob/master/example-notebooks/Demo.ipynb), which contains the correct metadata in the JSON in the .ipynb to get Colab to spin up an R session without. So, the notebook started blank (but R) and then the sleepwalk demo code was cut&pasted into it.

In [0]:
system("apt-get install libpoppler-cpp-dev")   # for antiwar
system("apt-get install libapparmor-dev")      # for unrtf

In [2]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install()


Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Old packages: 'digest', 'rlang', 'roxygen2', 'rprojroot', 'scales', 'selectr',
  'tidyverse', 'xtable'


In [3]:
BiocManager::valid()

Warning message:
“8 packages out-of-date; 0 packages too new”


* sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] BiocManager_1.30.10 compiler_3.6.1      IRdisplay_0.7.0    
 [4] pbdZMQ_0.3-3        tools_3.6.1         htmltools_0.4.0    
 [7] pillar_1.4.2        base64enc_0.1-3     crayon_1.3.4       
[10] Rcpp_1.0.3          uuid_0.1-2    

## Install sleepwalk

https://anders-biostat.github.io/sleepwalk/

In [4]:
install.packages( "sleepwalk" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


## Test drive sleepwalk

After successfully installing the code on Colab, let's take it for a test drive. There is a notebook in sleepwalk's repo, [Demo1.R](https://github.com/anders-biostat/sleepwalk/blob/master/test/demo1.R) which walks through the basic. This notebook takes that code and gets it running on Colab.

Some relevant links:
- [Rtsne on CRAN](https://cran.r-project.org/web/packages/Rtsne/index.html)

In [0]:
library(readtext)

In [8]:
# TODO: Installs are slow, so only install that which is not already installed.
# Via: https://stackoverflow.com/a/4090208

dependency_packages <- c("Rtsne", "irlba", "umap", "sleepwalk", "readtext")
uninstalled_packages <- dependency_packages[!(dependency_packages %in% installed.packages()[,"Package"])]
if(length(uninstalled_packages))
  { 
  cat(sprintf("Need to install %d packages.\n", length(uninstalled_packages)))
  install.packages(uninstalled_packages)
  } else { 
    print("No packages need to be installed", quote=FALSE)
    }

[1] No packages need to be installed


In [9]:
install.packages("readtext", verbose=TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
system (cmd0): /usr/lib/R/bin/R CMD INSTALL
foundpkgs: readtext, /tmp/RtmpTjeFeO/downloaded_packages/readtext_0.75.tar.gz
files: /tmp/RtmpTjeFeO/downloaded_packages/readtext_0.75.tar.gz
1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/RtmpTjeFeO/downloaded_packages/readtext_0.75.tar.gz'


In [10]:
library( Rtsne )
library( irlba )
library( umap )
library( sleepwalk ) 

Loading required package: Matrix


### Download some test data

From sleepwalk's [Demo1.R](https://github.com/anders-biostat/sleepwalk/blob/master/test/demo1.R):
```
# These two file can be downloaded from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE100866
countsRNA_filename <- "~/Downloads/GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv.gz"
countsADT_filename <- "~/Downloads/GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv.gz"
```
So let's grab those from [NIH](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE100866):

In [0]:
# /content/ is the default dir on Colab
countsRNA_URL <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE100866&format=file&file=GSE100866%5FCBMC%5F8K%5F13AB%5F10X%2DRNA%5Fumi%2Ecsv%2Egz"
countsRNA_filename <- "/content/GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv.gz"
                  
countsADT_URL <- "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE100866&format=file&file=GSE100866%5FCBMC%5F8K%5F13AB%5F10X%2DADT%5Fumi%2Ecsv%2Egz"
countsADT_filename <- "/content/GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv.gz"

download.file(countsRNA_URL, countsRNA_filename)
download.file(countsADT_URL, countsADT_filename)

In [12]:
# Check that we have files, especially files larger than zero bytes in size
print(list.files("/content/"))
print(file.size(list.files("/content/")))

[1] "GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv.gz"
[2] "GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv.gz"
[3] "output_of_sleepwalk"                      
[4] "output_of_sleepwalk.html"                 
[5] "sample_data"                              
[1]   194394 14871517  3373114  3373114     4096


In [0]:
# Load the file. (This takes a while as it is a large file.)
countsRNA <- as.matrix( read.csv( gzfile( countsRNA_filename ), row.names = 1) )
countsADT <- as.matrix( read.csv( gzfile( countsADT_filename ), row.names = 1) )


In [14]:
# Calculate for each cell ratio of molecules mapped to human genes
# versus mapped to mouse genes
human_mouse_ratio <- 
  colSums( countsRNA[ grepl( "HUMAN" , rownames(countsRNA) ), ] ) / 
  colSums( countsRNA[ grepl( "MOUSE" , rownames(countsRNA) ), ] )

# JFT debugging
cat(sprintf("countsRNA: %d\n" , length(countsRNA)))
cat(sprintf("countsADT: %d\n" , length(countsADT)))
cat(sprintf("# rownames: %d\n" , length(human_mouse_ratio)))


# Keep only the cells with at least 10 times more human than mouse genes
# and keep only the counts for the human genes.
countsRNA <- countsRNA[ 
  grepl( "HUMAN" , rownames(countsRNA) ), 
  human_mouse_ratio > 10 ]

# Remove the "HUMAN_" prefix from the gene names
rownames(countsRNA) <- sub( "HUMAN_", "", rownames(countsRNA) )

# JFT debugging
cat(sprintf("# colnames: %d\n", length(colnames(rownames))))

# Subset the ADT matrix to the same cells as in the RNA matrix
countsADT <- countsADT[ , colnames(countsRNA) ]

# Calculate size factors
exprsRNA <- matrix( nrow = nrow(countsRNA), ncol = ncol(countsRNA), dimnames = dimnames(countsRNA) )
for( j in seq.int( ncol(countsRNA) ) )
  exprsRNA[,j] <- log2( 1 + countsRNA[,j] / sum(countsRNA[,j]) )

# Run a PCA on the expression data
pca <- prcomp_irlba( t(exprsRNA), n=50 )

# Set the rownames manually (as IRLBA doesn't do that for us)
rownames(pca$x) <- colnames(exprsRNA)
rownames(pca$rotation) <- rownames(exprsRNA)

# Run t-SNE on the data
tsneRNA <- Rtsne( pca$x, pca=FALSE, verbose=TRUE )
rownames(tsneRNA$Y) <- rownames(pca$x)

countsRNA: 312624760
countsADT: 112021
# rownames: 8617
# colnames: 0
Read the 8005 x 50 data matrix successfully!
OpenMP is working. 1 threads.
Using no_dims = 2, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
Done in 3.22 seconds (sparsity = 0.017212)!
Learning embedding...
Iteration 50: error is 93.106630 (50 iterations in 2.59 seconds)
Iteration 100: error is 82.007539 (50 iterations in 2.21 seconds)
Iteration 150: error is 80.569511 (50 iterations in 1.68 seconds)
Iteration 200: error is 80.192435 (50 iterations in 1.75 seconds)
Iteration 250: error is 80.012220 (50 iterations in 1.80 seconds)
Iteration 300: error is 3.093761 (50 iterations in 1.55 seconds)
Iteration 350: error is 2.819440 (50 iterations in 1.40 seconds)
Iteration 400: error is 2.670648 (50 iterations in 1.35 seconds)
Iteration 450: error is 2.576804 (50 iterations in 1.34 seconds)
Iteration 500: error is 2.511833 (50 iterations in 1.33 seconds)
Iteration 550: error i

In [0]:

# Explore the result with sleepwalk
output_file_name <- "/content/output_of_sleepwalk.html"
sleepwalk( tsneRNA$Y, pca$x, 0.07, saveToFile=output_file_name)


In [16]:
print(list.files("/content/"))
print(file.size(list.files("/content/")))

[1] "GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv.gz"
[2] "GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv.gz"
[3] "output_of_sleepwalk"                      
[4] "output_of_sleepwalk.html"                 
[5] "sample_data"                              
[1]   194394 14871517  3373114  3381270     4096


In [0]:
#print(readtext(output_file_name))
file.show(output_file_name)
